In [1]:
!pip install -U phidata groq duckduckgo-search

In [8]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.model.groq import Groq
from phi.tools.duckduckgo import DuckDuckGo
import os

os.environ["GROQ_API_KEY"] = "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"

web_agent = Agent(
    name="Web Agent",
    model=Groq(id="llama3-groq-70b-8192-tool-use-preview"),
    tools=[DuckDuckGo()],
    instructions=["Always include sources"],
    show_tool_calls=True,
    markdown=True,
)
web_agent.print_response("What is your speciality?", stream=True)


Output()

In [13]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools

import os
os.environ["GROQ_API_KEY"] = "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"

finance_agent = Agent(
    name="Finance Agent",
    #model=OpenAIChat(id="gpt-4o"),
    model=Groq(id="llama3-groq-70b-8192-tool-use-preview"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True, company_news=True)],
    instructions=["Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)
finance_agent.print_response("Summarize analyst recommendations for NVDA. How much does it cost in today's date in INR", stream=True)


Output()

ERROR    Function convert_currency not found

In [23]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.yfinance import YFinanceTools

import os
os.environ["GROQ_API_KEY"] = "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"

web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=Groq(id="llama3-groq-70b-8192-tool-use-preview"),
    tools=[DuckDuckGo()],
    instructions=["Always include sources"],
    show_tool_calls=True,
    markdown=True,
)

finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=Groq(id="llama3-groq-70b-8192-tool-use-preview"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions=["Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

agent_team = Agent(
    team=[web_agent, finance_agent],
    model=Groq(id="llama3-groq-70b-8192-tool-use-preview"),
    instructions=["Always include sources", "Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

agent_team.print_response("Summarize analyst recommendations and share the latest news for NVDA", stream=True)


Output()

ERROR    Function search_lateest_news not found

In [26]:
pip install lancedb tantivy pypdf sqlalchemy


INFO: pip is looking at multiple versions of lancedb to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.0 MB/s eta 0:00:00


In [38]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.embedder.openai import OpenAIEmbedder
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.lancedb import LanceDb, SearchType
from lancedb import LanceDb, SearchType
from sentence_transformers import SentenceTransformer
from phi_knowledge_base import PDFUrlKnowledgeBase

import os
os.environ["GROQ_API_KEY"] = "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"

# Create a knowledge base from a PDF
class SentenceTransformerEmbedder:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed(self, texts):
        """
        Embed a list of texts into dense vectors.
        :param texts: List of strings to embed.
        :return: List of dense vector embeddings.
        """
        return self.model.encode(texts, convert_to_tensor=True).tolist()

# Initialize the knowledge base with SentenceTransformer Embedder
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=LanceDb(
        table_name="recipes",
        uri="tmp/lancedb",
        search_type=SearchType.vector,
        embedder=SentenceTransformerEmbedder(model_name="all-MiniLM-L6-v2"),
    ),
)


# Comment out after first run as the knowledge base is loaded
knowledge_base.load()

agent = Agent(

    # Add the knowledge base to the agent
    knowledge=knowledge_base,
    show_tool_calls=True,
    markdown=True,
)
agent.print_response("How do I make chicken and galangal in coconut milk soup", stream=True)


ImportError: cannot import name 'LanceDb' from 'lancedb' (/usr/local/lib/python3.10/dist-packages/lancedb/__init__.py)

In [59]:
from typing import List
from pydantic import BaseModel, Field
from phi.agent import Agent
from phi.model.groq import Groq

import os
os.environ["GROQ_API_KEY"] = "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"

# Define a Pydantic model to enforce the structure of the output
class MovieScript(BaseModel):
    setting: str = Field(..., description="Provide a nice setting for a blockbuster movie.")
    ending: str = Field(..., description="Ending of the movie. If not available, provide a happy ending.")
    genre: str = Field(..., description="Genre of the movie. If not available, select action, thriller or romantic comedy.")
    name: str = Field(..., description="Give a name to this movie")
    characters: List[str] = Field(..., description="Name of characters for this movie.")
    storyline: str = Field(..., description="3 sentence storyline for the movie. Make it exciting!")

# Agent that uses JSON mode
json_mode_agent = Agent(
    model=Groq(id="llama3-groq-70b-8192-tool-use-preview"),
    description="You write movie scripts.",
    response_model=MovieScript,
    response_format="json_object"
)
# Agent that uses structured outputs
structured_output_agent = Agent(
    model=Groq(id="llama3-groq-70b-8192-tool-use-preview"),
    description="You write movie scripts.",
    response_model=MovieScript,
    structured_outputs=True,
)

json_mode_agent.print_response(
    """
    Respond in strict JSON format:
    {
        "setting": "A bustling city with tall skyscrapers",
        "ending": "A thrilling car chase leads to a happy reunion",
        "genre": "Action",
        "name": "New York Chronicles",
        "characters": ["John", "Emma", "Mysterious Villain"],
        "storyline": "John and Emma team up to stop a mysterious villain from destroying the city."
    }

    Generate this for New York.
    """
)
structured_output_agent.print_response(
    """
    Respond in strict JSON format:
    {
        "setting": "A bustling city with tall skyscrapers",
        "ending": "A thrilling car chase leads to a happy reunion",
        "genre": "Action",
        "name": "New York Chronicles",
        "characters": ["John", "Emma", "Mysterious Villain"],
        "storyline": "John and Emma team up to stop a mysterious villain from destroying the city."
    }

    Generate this for New York.
    """
)


Output()

Output()

WARNING  Failed to convert response to pydantic model: 6 validation errors for MovieScript                         
         setting                                                                                                   
           Field required [type=missing, input_value={'movie': {'name': 'New Y...ds to a happy reunion'}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         ending                                                                                                    
           Field required [type=missing, input_value={'movie': {'name': 'New Y...ds to a happy reunion'}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         genre                                                                                                     
           Field required [type=missing, input_value={'movie': {'name': 'New Y...ds to a happy reunion'}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         name                                                                                                      
           Field required [type=missing, input_value={'movie': {'name': 'New Y...ds to a happy reunion'}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         characters                                                                                                
           Field required [type=missing, input_value={'movie': {'name': 'New Y...ds to a happy reunion'}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         storyline                                                                                                 
           Field required [type=missing, input_value={'movie': {'name': 'New Y...ds to a happy reunion'}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing

WARNING  Failed to convert response to response_model

In [57]:
# Use the agent to get a raw response
response = structured_output_agent.print_response("Generate a movie script for New York in JSON format.",)
print(response)


Output()

WARNING  Failed to convert response to pydantic model: 6 validation errors for MovieScript                         
         setting                                                                                                   
           Field required [type=missing, input_value={'movie': {'title': 'New ..., 'role': 'Comedian'}]}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         ending                                                                                                    
           Field required [type=missing, input_value={'movie': {'title': 'New ..., 'role': 'Comedian'}]}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         genre                                                                                                     
           Field required [type=missing, input_value={'movie': {'title': 'New ..., 'role': 'Comedian'}]}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         name                                                                                                      
           Field required [type=missing, input_value={'movie': {'title': 'New ..., 'role': 'Comedian'}]}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         characters                                                                                                
           Field required [type=missing, input_value={'movie': {'title': 'New ..., 'role': 'Comedian'}]}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing                              
         storyline                                                                                                 
           Field required [type=missing, input_value={'movie': {'title': 'New ..., 'role': 'Comedian'}]}},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing

WARNING  Failed to convert response to response_model

None


In [61]:
from phi.agent import Agent
from phi.model.groq import Groq

task = (
    "Three missionaries and three cannibals need to cross a river. "
    "They have a boat that can carry up to two people at a time. "
    "If, at any time, the cannibals outnumber the missionaries on either side of the river, the cannibals will eat the missionaries. "
    "How can all six people get across the river safely? Provide a step-by-step solution and show the solutions as an ascii diagram"
)

reasoning_agent = Agent(model=Groq(id="llama3-groq-70b-8192-tool-use-preview"), reasoning=True, markdown=True, structured_outputs=True)
reasoning_agent.print_response(task+"json", stream=True, show_full_reasoning=True)


Output()

WARNING  Failed to convert response to pydantic model: 1 validation error for ReasoningSteps                       
         reasoning_steps                                                                                           
           Field required [type=missing, input_value={'title': 'Crossing the R...'}], 'confidence': 0.95},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.10/v/missing

WARNING  Failed to convert response to response_model

ERROR    Reasoning error: 'str' object has no attribute 'reasoning_steps'

In [72]:
from phi.agent import Agent

agent = Agent(model=Groq(id="llama3-groq-70b-8192-tool-use-preview"), markdown=True, debug_mode=True, monitoring=True)
agent.print_response("Share a 20 sentence horror story")


DEBUG    Debug logs enabled

DEBUG    *********** Agent Run Start: fc486c09-b540-4273-94de-afef178c0fc3 ***********

DEBUG    ---------- Groq Response Start ----------

DEBUG    ============== system ==============

DEBUG    ## Instructions                                                                                           
         Use markdown to format your answers.

...

DEBUG    ---------- Groq Response End ----------

DEBUG    Added 2 Messages to AgentMemory

DEBUG    Added AgentRun to AgentMemory

DEBUG    --**-- Logging Agent Run

DEBUG    *********** Agent Run End: fc486c09-b540-4273-94de-afef178c0fc3 ***********